In [1]:
!wget https://www.mediafire.com/file_premium/szd8k40iyfcs01z/siamese_curated.zip/file

--2024-02-23 15:47:43--  https://www.mediafire.com/file_premium/szd8k40iyfcs01z/siamese_curated.zip/file
Resolving www.mediafire.com (www.mediafire.com)... 104.16.114.74, 104.16.113.74
Connecting to www.mediafire.com (www.mediafire.com)|104.16.114.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://download1075.mediafire.com/ua93e1e95nfg_QEhTT2IN_ygLos1pflcDBFoh4Bs7k0KRGuc5X6AzmBOZ9YeIptoiIoFSwL4ZKkHm9iolXBIg7hMuzgDPpZFxlF54riWhs5s0q-z0rQP7h0SEqxefaGeHS0jKhe-T_QkrpXmAy12iJzRhrdIjtoHvcC8-tL9BAQEfi-y/szd8k40iyfcs01z/siamese_curated.zip [following]
--2024-02-23 15:47:43--  https://download1075.mediafire.com/ua93e1e95nfg_QEhTT2IN_ygLos1pflcDBFoh4Bs7k0KRGuc5X6AzmBOZ9YeIptoiIoFSwL4ZKkHm9iolXBIg7hMuzgDPpZFxlF54riWhs5s0q-z0rQP7h0SEqxefaGeHS0jKhe-T_QkrpXmAy12iJzRhrdIjtoHvcC8-tL9BAQEfi-y/szd8k40iyfcs01z/siamese_curated.zip
Resolving download1075.mediafire.com (download1075.mediafire.com)... 205.196.122.16
Connecting to download1075.mediafire.com (download107

In [2]:
!unzip file

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: siamese_curated/670bc75a3eca98468a50216bed69e63d49edbe1b3f613e4dee94927f2c4a740c.jpg  
  inflating: siamese_curated/b31e5d0f68d949f3604211fdbb055426518e6bbee73e59e19a3b5764f58975f4.jpg  
  inflating: siamese_curated/5be528f787fdc0200457426c242d84a32e2070b8c2e4cdb71d68b71c308b7eb2.jpg  
  inflating: siamese_curated/11f176e92f90581333f5059d9e8c739d80011df5e53ec42d808b3ea412910bb5.jpg  
  inflating: siamese_curated/874c356371783ed19811d623c8d66ddde51af0641e20a7a8da57091b1f1e9621.jpg  
  inflating: siamese_curated/22485707f1495fd1452df138a4d933fb65f2248001e788cdd1088099494a2895.jpg  
  inflating: siamese_curated/1d8b992ae4eb8e34e1750fd9eaabce5ffd384946ff5bbbff64a6579815a9517a.jpg  
  inflating: siamese_curated/7b85b709de8c5432fb76c17ce4ce93cc25bb636a689e36d25fc42ed043146bf3.jpg  
  inflating: siamese_curated/3c3885e421781a29af13fa3de8bb8c03021cdaabc7e5f7f275c62fd772c48171.jpg  
  inflating: siamese_curated/6d1f

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import random
import os
import torch
from collections import defaultdict

random.seed(42)
torch.manual_seed(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class ImagePairsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = self.load_data(f'{root_dir}/{csv_file}')
        self.root_dir = root_dir
        self.transform = transform
        self.d = self.load_dict()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        (img1_path, video1, interval1) = self.data[idx]
        remaining_keys = list(self.d[video1].keys())
        picked_key = random.choice(remaining_keys)
        img2_path = random.choice(self.d[video1][picked_key])
        similar = int(picked_key == interval1)

        # Load images
        img1 = Image.open(os.path.join(self.root_dir, img1_path)).convert('RGB')
        img2 = Image.open(os.path.join(self.root_dir, img2_path)).convert('RGB')

        # Apply transformations if provided
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, similar

    def load_data(self, csv_file):
        data = []
        with open(csv_file, 'r') as file:
            for line in file:
                filename, video, interval = line.strip().split(',')
                data.append((filename, video, interval))
        return data

    def load_dict(self):
        di = defaultdict(lambda: defaultdict(list))
        for data in self.data:
            path, video, interval = data
            di[video][interval].append(path)
        return di


class ImagePairsCuratedDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, tag='train'):
        self.tag = tag
        self.data = self.load_data(f'{root_dir}/{csv_file}')
        self.root_dir = root_dir
        self.transform = transform
        self.d = self.load_dict()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        (img1_path, video1, interval1) = self.data[idx]
        remaining_keys = list(self.d[video1].keys())
        picked_key = random.choice(remaining_keys)
        img2_path = random.choice(self.d[video1][picked_key])
        similar = int(picked_key == interval1)

        # Load images
        img1 = Image.open(os.path.join(self.root_dir, img1_path)).convert('RGB')
        img2 = Image.open(os.path.join(self.root_dir, img2_path)).convert('RGB')

        # Apply transformations if provided
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, similar

    def load_data(self, csv_file):
        data = []
        with open(csv_file, 'r') as file:
            for line in file:
                filename, tag, video, interval, _ = line.strip().split(',')
                if self.tag == tag:
                  data.append((filename, video, interval))
        return data

    def load_dict(self):
        di = defaultdict(lambda: defaultdict(list))
        for data in self.data:
            path, video, interval = data
            di[video][interval].append(path)
        return di

# Example usage:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset = ImagePairsCuratedDataset('labels.csv', 'siamese_curated', transform=transform)
validation_dataset = ImagePairsCuratedDataset('labels.csv', 'siamese_curated', transform=transform, tag='test')


In [33]:
batch_size=16
# Create DataLoader instances using the defined samplers
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

In [8]:
ones = 0
for img1, img2, label in train_loader:
  l = list(label)
  ones += sum(l)
  print(ones)
print(ones/ len(train_loader))



tensor(2)
tensor(5)
tensor(9)
tensor(10)
tensor(11)
tensor(14)
tensor(17)
tensor(21)
tensor(23)
tensor(26)
tensor(29)
tensor(34)
tensor(34)
tensor(35)
tensor(41)
tensor(42)
tensor(46)
tensor(48)
tensor(52)
tensor(53)
tensor(58)
tensor(61)
tensor(61)
tensor(64)
tensor(66)
tensor(70)
tensor(75)
tensor(77)
tensor(78)
tensor(79)
tensor(83)
tensor(85)
tensor(88)
tensor(90)
tensor(91)
tensor(92)
tensor(94)
tensor(97)
tensor(97)
tensor(98)
tensor(98)
tensor(102)
tensor(104)
tensor(104)
tensor(109)
tensor(110)
tensor(115)
tensor(117)
tensor(118)
tensor(122)
tensor(123)
tensor(125)
tensor(125)
tensor(130)
tensor(133)
tensor(136)
tensor(140)
tensor(141)
tensor(143)
tensor(143)
tensor(148)
tensor(151)
tensor(152)
tensor(156)
tensor(158)
tensor(158)
tensor(161)
tensor(162)
tensor(164)
tensor(171)
tensor(175)
tensor(178)
tensor(179)
tensor(183)
tensor(184)
tensor(186)
tensor(186)
tensor(191)
tensor(195)
tensor(196)
tensor(196)
tensor(201)
tensor(203)
tensor(205)
tensor(209)
tensor(212)
tensor(215)


In [82]:
from torch import nn
from torch import flatten
from torch.functional import F
import numpy as np


class SiameseNetwork(nn.Module):
    def __init__(self, resnet_model):
        super(SiameseNetwork, self).__init__()

        # Use the same ResNet model for both branches
        self.resnet_branch1 = resnet_model

        # Additional layers for prediction
        self.fc = nn.Linear(512, 400)
        self.fc2 = nn.Linear(400, 128)


    def forward_one_branch(self, x):
        x = self.resnet_branch1(x)
        x = F.leaky_relu(x)
        x = flatten(x, 1)
        x = self.fc(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        return x

    def forward(self, input1, input2):
        # Forward pass for each branch
        x1 = self.forward_one_branch(input1)
        x2 = self.forward_one_branch(input2)
        return x1,x2


class SiameseMobileNetwork(nn.Module):
    def __init__(self, mobilenet):
        super(SiameseMobileNetwork, self).__init__()

        # Use the same ResNet model for both branches
        self.mobilenet = mobilenet

        # Additional layers for prediction
        self.fc = nn.Linear(1000, 512)
        self.fc2 = nn.Linear(512, 128)


    def forward_one_branch(self, x):
        x = self.mobilenet(x)
        x = F.leaky_relu(x)
        x = flatten(x, 1)
        x = self.fc(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        return x

    def forward(self, input1, input2):
        # Forward pass for each branch
        x1 = self.forward_one_branch(input1)
        x2 = self.forward_one_branch(input2)
        return x1,x2


class SiameseYoloNetwork(nn.Module):
    def __init__(self, yolo):
        super(SiameseYoloNetwork, self).__init__()

        # Use the same ResNet model for both branches
        self.yolo = yolo

        # Additional layers for prediction
        self.fc = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 200)
        self.fc3 = nn.Linear(200, 128)


    def forward_one_branch(self, x):
        x = self.yolo.embed(x, verbose=False)
        x = list(map(lambda y: y.unsqueeze(0), x))
        x = torch.cat(x, dim=0)
        x = self.fc(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        return x

    def forward(self, input1, input2):
        # Forward pass for each branch
        x1 = self.forward_one_branch(input1)
        x2 = self.forward_one_branch(input2)
        return x1,x2

In [74]:
import torch.nn.functional as F


class ContrastiveLoss(nn.Module):
    def __init__(self):
        super(ContrastiveLoss, self).__init__()

    def forward(self, x1, x2, labels):
      cosine_similarity = F.cosine_similarity(x1, x2)
      loss = torch.mean(
          0.86 * (labels * torch.pow(1 - cosine_similarity, 2)) +
          0.14 * ((1 - labels) * torch.pow(cosine_similarity * (cosine_similarity + 1), 2))
      )
      return loss


class ContrastiveEuclideanLoss(nn.Module):
    def __init__(self, margin):
        self.margin = margin
        super(ContrastiveEuclideanLoss, self).__init__()
    def forward(self, x1, x2, labels):
        euclidean_distance = F.pairwise_distance(x1, x2)
        loss = torch.mean(
            0.86 * (labels * euclidean_distance ** 2) +
            0.14 * ((1 - labels) * torch.clamp(self.margin - euclidean_distance, min=0.0) ** 2)
        )
        return loss


In [9]:
print(len(train_loader))
print(len(test_loader))

510
167


In [56]:
def train(optimizer, criterion, patience, model, num_epochs, model_name):
  min_loss = float('inf')
  patience_counter = 0
  for epoch in range(num_epochs):
      running_loss = 0
      for index, batch in enumerate(train_loader):
          img1, img2, labels = batch
          img1 = img1.to(device)
          img2 = img2.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()

          # Forward pass
          o1, o2 = model(img1, img2)

          # Compute contrastive loss
          loss = criterion(o1, o2, labels.float())
          # Backward pass and optimization
          loss.backward()
          optimizer.step()

          running_loss += loss.detach()

          print(f'Batch {index + 1} of {len(train_loader)}', running_loss / (index+1))
      # Validation
      val_loss = 0.0
      with torch.no_grad():
          for batch in test_loader:
              img1, img2, labels = batch
              img1 = img1.to(device)
              img2 = img2.to(device)
              labels = labels.to(device)
              o1, o2 = model(img1, img2)
              val_loss += criterion(o1, o2, labels.float()).item()
      if val_loss < min_loss:
          min_loss = val_loss
          torch.save(model.state_dict(), f'drive/MyDrive/{model_name}_{val_loss}.pth')
          patience_counter = 0
      else:
          patience_counter += 1
      if patience_counter == patience:
          break
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Val Loss: {val_loss/len(test_loader)}')

In [20]:
from torch.optim import SGD


import torchvision.models as models

resnet_model = models.resnet18(pretrained=True)
resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])
for child in resnet_model.children():
    for param in child.parameters():
        param.requires_grad = True
resnet_model.train()

siamese_resnet1 = SiameseNetwork(resnet_model).to(device)
# Use contrastive loss as the criterion
criterion = ContrastiveLoss().to(device)

# Use Adam optimizer
optimizer = SGD(siamese_resnet1.parameters(), lr=0.001)




train(optimizer, criterion, 20, siamese_resnet1, 100, 'siameseResnetCuratedCosine')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Batch 110 of 510 tensor(0.0432, device='cuda:0')
Batch 111 of 510 tensor(0.0430, device='cuda:0')
Batch 112 of 510 tensor(0.0429, device='cuda:0')
Batch 113 of 510 tensor(0.0432, device='cuda:0')
Batch 114 of 510 tensor(0.0436, device='cuda:0')
Batch 115 of 510 tensor(0.0436, device='cuda:0')
Batch 116 of 510 tensor(0.0434, device='cuda:0')
Batch 117 of 510 tensor(0.0432, device='cuda:0')
Batch 118 of 510 tensor(0.0430, device='cuda:0')
Batch 119 of 510 tensor(0.0431, device='cuda:0')
Batch 120 of 510 tensor(0.0428, device='cuda:0')
Batch 121 of 510 tensor(0.0427, device='cuda:0')
Batch 122 of 510 tensor(0.0431, device='cuda:0')
Batch 123 of 510 tensor(0.0430, device='cuda:0')
Batch 124 of 510 tensor(0.0431, device='cuda:0')
Batch 125 of 510 tensor(0.0428, device='cuda:0')
Batch 126 of 510 tensor(0.0434, device='cuda:0')
Batch 127 of 510 tensor(0.0434, device='cuda:0')
Batch 128 of 510 tensor(0.0432, device='cuda:0')
Ba

In [ ]:
from torch.optim import SGD


import torchvision.models as models

resnet_model = models.resnet18(pretrained=True)
resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])
for child in resnet_model.children():
    for param in child.parameters():
        param.requires_grad = True
resnet_model.train()

siamese_resnet2 = SiameseNetwork(resnet_model).to(device)
# Use contrastive loss as the criterion
criterion = ContrastiveEuclideanLoss(1).to(device)

# Use Adam optimizer
optimizer = SGD(siamese_resnet2.parameters(), lr=0.001)




train(optimizer, criterion, 20, siamese_resnet2, 100, 'siameseResnetCuratedEuclidea')

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Batch 348 of 510 tensor(0.0301, device='cuda:0')
Batch 349 of 510 tensor(0.0301, device='cuda:0')
Batch 350 of 510 tensor(0.0301, device='cuda:0')
Batch 351 of 510 tensor(0.0301, device='cuda:0')
Batch 352 of 510 tensor(0.0302, device='cuda:0')
Batch 353 of 510 tensor(0.0302, device='cuda:0')
Batch 354 of 510 tensor(0.0302, device='cuda:0')
Batch 355 of 510 tensor(0.0303, device='cuda:0')
Batch 356 of 510 tensor(0.0303, device='cuda:0')
Batch 357 of 510 tensor(0.0303, device='cuda:0')
Batch 358 of 510 tensor(0.0303, device='cuda:0')
Batch 359 of 510 tensor(0.0302, device='cuda:0')
Batch 360 of 510 tensor(0.0303, device='cuda:0')
Batch 361 of 510 tensor(0.0302, device='cuda:0')
Batch 362 of 510 tensor(0.0302, device='cuda:0')
Batch 363 of 510 tensor(0.0302, device='cuda:0')
Batch 364 of 510 tensor(0.0301, device='cuda:0')
Batch 365 of 510 tensor(0.0301, device='cuda:0')
Batch 366 of 510 tensor(0.0302, device='cuda:0')
Ba

In [23]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.0/716.0 kB 10.5 MB/s eta 0:00:00


In [ ]:
import torchvision.models as models
import torch
from ultralytics import YOLO

model = YOLO('best_rico.pt')
siamese_mobile_net = SiameseYoloNetwork(model).to(device)
# Use contrastive loss as the criterion
criterion = ContrastiveLoss().to(device)

# Use Adam optimizer
optimizer = SGD(siamese_mobile_net.parameters(), lr=0.001)

train(optimizer, criterion, 20, siamese_mobile_net, 100, 'siameseyolouratedCosine')

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Batch 458 of 510 tensor(0.0535, device='cuda:0')
Batch 459 of 510 tensor(0.0534, device='cuda:0')
Batch 460 of 510 tensor(0.0536, device='cuda:0')
Batch 461 of 510 tensor(0.0536, device='cuda:0')
Batch 462 of 510 tensor(0.0536, device='cuda:0')
Batch 463 of 510 tensor(0.0535, device='cuda:0')
Batch 464 of 510 tensor(0.0535, device='cuda:0')
Batch 465 of 510 tensor(0.0534, device='cuda:0')
Batch 466 of 510 tensor(0.0535, device='cuda:0')
Batch 467 of 510 tensor(0.0535, device='cuda:0')
Batch 468 of 510 tensor(0.0535, device='cuda:0')
Batch 469 of 510 tensor(0.0535, device='cuda:0')
Batch 470 of 510 tensor(0.0534, device='cuda:0')
Batch 471 of 510 tensor(0.0535, device='cuda:0')
Batch 472 of 510 tensor(0.0536, device='cuda:0')
Batch 473 of 510 tensor(0.0536, device='cuda:0')
Batch 474 of 510 tensor(0.0535, device='cuda:0')
Batch 475 of 510 tensor(0.0535, device='cuda:0')
Batch 476 of 510 tensor(0.0535, device='cuda:0')
Ba

In [ ]:
model = YOLO('best_rico.pt')
siamese_mobile_net = SiameseYoloNetwork(model).to(device)
# Use contrastive loss as the criterion
criterion = ContrastiveEuclideanLoss(1).to(device)

# Use Adam optimizer
optimizer = SGD(siamese_mobile_net.parameters(), lr=0.001)

train(optimizer, criterion, 20, siamese_mobile_net, 100, 'siameseyolouratedEuclidean')

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Batch 399 of 510 tensor(0.0481, device='cuda:0')
Batch 400 of 510 tensor(0.0481, device='cuda:0')
Batch 401 of 510 tensor(0.0481, device='cuda:0')
Batch 402 of 510 tensor(0.0480, device='cuda:0')
Batch 403 of 510 tensor(0.0481, device='cuda:0')
Batch 404 of 510 tensor(0.0481, device='cuda:0')
Batch 405 of 510 tensor(0.0481, device='cuda:0')
Batch 406 of 510 tensor(0.0481, device='cuda:0')
Batch 407 of 510 tensor(0.0481, device='cuda:0')
Batch 408 of 510 tensor(0.0481, device='cuda:0')
Batch 409 of 510 tensor(0.0482, device='cuda:0')
Batch 410 of 510 tensor(0.0482, device='cuda:0')
Batch 411 of 510 tensor(0.0482, device='cuda:0')
Batch 412 of 510 tensor(0.0482, device='cuda:0')
Batch 413 of 510 tensor(0.0482, device='cuda:0')
Batch 414 of 510 tensor(0.0481, device='cuda:0')
Batch 415 of 510 tensor(0.0481, device='cuda:0')
Batch 416 of 510 tensor(0.0480, device='cuda:0')
Batch 417 of 510 tensor(0.0480, device='cuda:0')
Ba

In [ ]:
import torchvision.models as models
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.train()
for child in model.children():
    for param in child.parameters():
        param.requires_grad = True

siamese_mobile_net2 = SiameseMobileNetwork(model).to(device)
# Use contrastive loss as the criterion
criterion = ContrastiveEuclideanLoss(1).to(device)

# Use Adam optimizer
optimizer = SGD(siamese_mobile_net2.parameters(), lr=0.001)

train(optimizer, criterion, 20, siamese_mobile_net2, 100, 'siameseMobilenetCuratedEuclidean')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Batch 417 of 510 tensor(0.0469, device='cuda:0')
Batch 418 of 510 tensor(0.0469, device='cuda:0')
Batch 419 of 510 tensor(0.0470, device='cuda:0')
Batch 420 of 510 tensor(0.0470, device='cuda:0')
Batch 421 of 510 tensor(0.0470, device='cuda:0')
Batch 422 of 510 tensor(0.0470, device='cuda:0')
Batch 423 of 510 tensor(0.0469, device='cuda:0')
Batch 424 of 510 tensor(0.0469, device='cuda:0')
Batch 425 of 510 tensor(0.0469, device='cuda:0')
Batch 426 of 510 tensor(0.0471, device='cuda:0')
Batch 427 of 510 tensor(0.0470, device='cuda:0')
Batch 428 of 510 tensor(0.0470, device='cuda:0')
Batch 429 of 510 tensor(0.0470, device='cuda:0')
Batch 430 of 510 tensor(0.0470, device='cuda:0')
Batch 431 of 510 tensor(0.0470, device='cuda:0')
Batch 432 of 510 tensor(0.0469, device='cuda:0')
Batch 433 of 510 tensor(0.0469, device='cuda:0')
Batch 434 of 510 tensor(0.0469, device='cuda:0')
Batch 435 of 510 tensor(0.0469, device='cuda:0')
Ba